# SMPLX-basic

SMPLX extends SMPL with fully articulated hands and an expressive face.

In this notebook, we will introduce the basic usage of `smplx.SMPLX` model, and some common problems you may encounter.

And there is a "SMPL family", which is a set of models related to SMPL. You can have a overview of them in [SMPL wiki](https://meshcapade.wiki/SMPL).

The basic usage of SMPLX is quite like SMPL, so we will be brief in the basic concept, and mainly focus on the differences between them. However, the **data of SMPL and SMPLX are quite different** (from the inputs to the outputs), you shouldn't mix them up.

## Preparation

You should also prepare the environments for SMPL, check the [SMPL-basic](./SMPL_basic.ipynb) for more details.

We use three different checkpoints: `SMPLX_NEUTRAL.npz`, `SMPLX_MALE.npz` and `SMPLX_FEMALE.npz`. You can get them from [SMPL eXpressive](https://smpl-x.is.tue.mpg.de/).

After downloading the SMPLX's checkpoints, you should put them to `data_inputs/body_models/smplx`, your directory tree should look like this:

```
.
├── SMPLX_FEMALE.npz
├── SMPLX_MALE.npz
└── SMPLX_NEUTRAL.npz
```

## Tutorials

### Environment Preparation

In [ ]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL, SMPLX

# Things you don't need to care about. They are just for driving the tutorials.
from lib.path_manager import PathManager
from ez4d.vis.wis3d import HWis3D as Wis3D
from ez4d.kinematics.abstract_skeletons import Skeleton_SMPL22

pm = PathManager()

### Load SMPLX model

In [16]:
B = 150

body_models = {}
genders = ['neutral', 'female', 'male']  # case insensitive

for gender in genders:
    body_models[gender] = SMPLX(
            model_path = pm.inputs / 'body_models' / 'smplx',
            gender     = gender,
            batch_size = 150,
        )

# Prepare some parameters for later inference.
body_model : SMPLX = body_models['neutral']  # use neutral for example

# Prepare mesh template for later visualization.
# Tips: mesh = vertices + faces, and the faces are the indices of vertices, which won't change across SMPL's outputs.
mesh_temp : np.ndarray = body_model.faces  # (20908, 3)

We can find that, the shape of `faces` changes. Which means the **vertices of SMPLX model is different from SMPL model's**. We will dive into this later.

Another thing you should notice is that, the implement of `smplx.SMPLX` does not support dynamic batch inference in through it's API. You need to define the batch size while defining the model. 

However, you still can make batch inference possible through modify the implements of the API, but those are not covered in this notebook.

### SMPL-X Inference

In [4]:
# Inference.
smplx_out = body_model(
        betas         = torch.zeros(B, 10),
        global_orient = torch.zeros(B, 3),
        body_pose     = torch.zeros(B, 63),
        transl        = torch.zeros(B, 3),
    )

# Check output.
joints : torch.Tensor = smplx_out.joints    # (B, 127, 3)
verts  : torch.Tensor = smplx_out.vertices  # (B, 10475, 3)
print(joints.shape, verts.shape)

torch.Size([150, 127, 3]) torch.Size([150, 10475, 3])


I want to highlight that, the input format of `smplx.SMPLX` is not similar to `smplx.SMPL`'s. For example, the shape of the tensor inputted to `body_pose`, changes from `(B, J, 3)` to `(B, J'*3)`, where `J = 23` and `J' = 21`, so does the `global_orient`, which changes from `(B, 1, 3)` to `(B, 1*3)`.

> The reason why `J'` decreases to 21 is that, SMPL-X decouples the pose of hands from body pose to `left_hand_pose` and `right_hand_pose`, so that it can express more complex hand movements. Check the paper for details if you are interested. Hands are not what we are going to talk about here.

One parameter I want to address is `transl`. You shouldn't share translation between SMPL and SMPLX, just like other parameters.

In [5]:
smplx_wis3d = Wis3D(
        pm.outputs / 'wis3d',
        'SMPLX',
    )

smplx_wis3d.add_motion_verts(
        verts  = verts[:1],
        name   = f'smplx_T_pose',
        offset = 0,
    )

In [6]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

Serving on http://0.0.0.0:19090
^C


### SMPL-X v.s. SMPL

Not only the input format changes, but also the output format changes. The `vertices` of SMPL-X model is different from SMPL model's. The number of vertices of SMPL-X model is `10475`, while SMPL model's is `6890`.

And you can see that the T-Pose of SMPL-X model is more 'relax' than SMPL model's. Now let's put them together to see the differences.

In [11]:
# Generate the SMPL T-pose.
body_model_smpl = SMPL( model_path=pm.inputs/'body_models'/'smpl', gender="neutral")
smpl_out = body_model_smpl()
smpl_verts = smpl_out.vertices
smpl_joints = smpl_out.joints

In [14]:
smpl_vs_smplx_wis3d = Wis3D(
        pm.outputs / 'wis3d',
        'SMPL v.s. SMPLX',
    )

# Visualize SMPL results.
smpl_vs_smplx_wis3d.add_motion_verts(
    verts  = smpl_verts[:1],
    name   = f'smpl_T_pose',
    offset = 0,
)

smpl_vs_smplx_wis3d.add_motion_skel(
    joints = smpl_joints[:1, :22],
    bones  = Skeleton_SMPL22.bones,
    colors = Skeleton_SMPL22.bone_colors,
    name   = f'smpl_T_pose',
    offset = 0,
)

# Visualize SMPL-X results.
smpl_vs_smplx_wis3d.add_motion_verts(
    verts  = verts[:1],
    name   = f'smplx_T_pose',
    offset = 0,
)

smpl_vs_smplx_wis3d.add_motion_skel(
    joints = joints[:1, :22],
    bones  = Skeleton_SMPL22.bones,
    colors = Skeleton_SMPL22.bone_colors,
    name   = f'smplx_T_pose',
    offset = 0,
)

In [15]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

Serving on http://0.0.0.0:19090
^C


## Additional Resources / Questions & Answers

1. Can I transfer the parameters between SMPL, SMPL-H and SMPL-X?
   - Yes, check the URLs below:
     - https://github.com/vchoutas/smplx/blob/main/transfer_model/README.md